In [1]:
import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from tensorboardX import SummaryWriter
writer = SummaryWriter(log_dir='./logs')
import numpy as np
from sklearn.model_selection import train_test_split
import torch.utils.data as Data
import os
import json


In [2]:
# X_train = torch.from_numpy(X_train)
# y_train = torch.from_numpy(y_train)
# X_test = torch.from_numpy(X_test)
# y_test = torch.from_numpy(y_test)
# x_y_dataset = Data.TensorDataset(X_train, y_train)
# test_x_y_dataset = Data.TensorDataset(X_test, y_test)

In [3]:
import Tool.tool_for_cut_text as tool_for_cut_text
import gensim.models.word2vec as w2v
import Tool.similarity_sentence as similarity_sentence

model = w2v.Word2Vec.load('C:/shu_item/STEO2/model7/model7.model')

In [4]:
def get_type_info():
    cwd = os.getcwd()
    real_path = os.getcwd() + '\experiment1_keywords_type'
    os.chdir(real_path)
    types = os.listdir()
    # 还原路径
    os.chdir(cwd)

    return [[type, type, get_events_len_by_path(real_path + '\\' + type + '\\' + type + '.txt'),real_path + '\\' + type + '\\' + type + '.txt'] for type in types]


def get_events_len_by_path(path):
    print(path)
    events = list()
    event = ''
    with open(path, 'r', encoding='utf8') as f:
        for line in f:
            if line != '\n':
                event += line
            else:
                if event != '':
                    events.append(event)
                    event = ''
    return len(events)

def get_events_by_path(path,type):
    events = list()
    event = ''
    with open(path, 'r', encoding='utf8') as f:
        for line in f:
            if line != '\n':
                event += line
            else:
                if event != '':
                    events.append([event,type])
                    event = ''
    return events

In [5]:
type_info = get_type_info()

C:\shu_item\pytorch-handbook\RNNItem\word2vecForRNN\experiment1_keywords_type\军事\军事.txt
C:\shu_item\pytorch-handbook\RNNItem\word2vecForRNN\experiment1_keywords_type\娱乐\娱乐.txt
C:\shu_item\pytorch-handbook\RNNItem\word2vecForRNN\experiment1_keywords_type\科技\科技.txt


In [6]:
# 提取 事件文本，类别 存入event_data
event_data = list()
for info in type_info:
    event_data.extend(get_events_by_path(info[3],info[1]))

In [7]:
event_data[444]

['仅需36分钟 天津将建到北京大兴国际机场高铁\n2019-02-25 12:33:51\n今年9月就将通航的北京大兴国际机场，把更广阔的腹地延伸到了京津冀三地。中国铁路设计集团近日（21日）对天津至北京大兴国际机场联络线进行第一次环评公示。记者注意到，未来天津至北京大兴国际机场的联络线跨越京津冀三地，为高速铁路。根据铁路有关部门的规划，这条线路预计今年开工，建成后从天津到北京新机场的时间约36分钟。近日，天津至北京大兴国际机场联络线环境影响评价首次信息公示。今年1月，河北省发改委也发布消息称，新建铁路天津至北京新机场联络线获批，批文显示，新建铁路天津至北京新机场联络线起自天津西站，利用津保铁路至胜芳站，由胜芳站新建线路引出，经安次区、永清县引入京雄城际固安东站，利用京雄城际至北京大兴国际机场。\n',
 '科技']

In [8]:
# 类别映射为数字
def type_encoding(type_info):
    type_num_dict = {}
    for i,type_name in enumerate(type_info):
        type_num_dict[type_name[1]] = i
    return type_num_dict

type_num_dict = type_encoding(type_info)
print(type_num_dict)

{'军事': 0, '娱乐': 1, '科技': 2}


In [9]:
# 存入每个事件的时间序列
event_series_type = []
for event_info in event_data:
    event_series = similarity_sentence.get_sentence_words_vector_to_rnn(event_info[0][:200],model)
    event_type = type_num_dict[event_info[1]]
    event_series_type.append([event_series,event_type])
    

In [12]:
event_series_type[440][0].shape

(151, 100)

In [13]:
events_series = list()
events_type = list()
for item in event_series_type:
    events_series.append(torch.from_numpy(item[0]).float())
    events_type.append(item[1])

In [14]:
events_series[88].shape

torch.Size([163, 100])

In [15]:
# np.save('event_series',evnet_series)
# np.save('event_type',event_type)

In [16]:
# event_series_load = np.load('event_series.npy',allow_pickle=True)
# event_type_load = np.load('event_type.npy',allow_pickle=True)

In [17]:
events_series_pad = rnn_utils.pad_sequence(events_series,batch_first=True)

In [18]:
events_series_pad.shape

torch.Size([580, 172, 100])

In [19]:
train = events_series_pad.numpy()
labels = np.array(events_type)

X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.2)

X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).long()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()
x_y_dataset = Data.TensorDataset(X_train, y_train)
test_x_y_dataset = Data.TensorDataset(X_test, y_test)

In [20]:
BATCH_SIZE = 32
train_loader = Data.DataLoader(dataset=x_y_dataset, batch_size=BATCH_SIZE,shuffle=True,num_workers=4)
test_loader = Data.DataLoader(dataset=test_x_y_dataset, batch_size=BATCH_SIZE,shuffle=False,num_workers=4)

In [27]:
class RNN(nn.Module):
    def __init__(self,input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(         # if use nn.RNN(), it hardly learns
            input_size=input_size,
            hidden_size=hidden_size,         # rnn hidden unit
            num_layers=2,           # number of rnn layer
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
            dropout=0.5,
        )

        self.out = nn.Linear(hidden_size, 64)
        self.out2 = nn.Linear(64,output_size)

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        r_out, (h_n, h_c) = self.rnn(x, None)   # None represents zero initial hidden state

        # choose r_out at the last time step
        out = self.out(r_out[:, -1, :])
        out = self.out2(out)
        return out

n_hidden = 128
LR = 0.01


rnn = RNN(100, n_hidden, 3)
rnn = rnn.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=LR)   # optimize all cnn parameters

In [28]:
def train(model, device, train_loader, optimizer, loss_func, epoch):
    model.train()
    for step, (data, target) in enumerate(train_loader):  # gives batch data, normalize x when iterate train_loader
        data, target = data.to(device), target.to(device)
        output = model(data)  # model output
        output = output.to(device)
        loss = loss_func(output, target)  # cross entropy loss
        optimizer.zero_grad()  # clear gradients for this training step
        loss.backward()  # backpropagation, compute gradients
        optimizer.step()  # apply gradients
        pred_y = output.max(1, keepdim=True)[1]  # 找到概率最大的下标
        correct = pred_y.eq(target.view_as(pred_y)).sum().item()
        accuracy = correct / len(target)
    
    writer.add_scalar(graph_name+'/train accuracy', accuracy, epoch)
    writer.add_scalar(graph_name+'/train loss',loss.item(),epoch)


def model_test(model, device, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred_y = output.max(1, keepdim=True)[1]  # 找到概率最大的下标
            correct += pred_y.eq(target.view_as(pred_y)).sum().item()

    test_len = len(test_loader.dataset)
    accuracy = correct / test_len
    writer.add_scalar(graph_name +'/test accuracy', accuracy, epoch)

    print("model_test data accuracy: %.9f" % accuracy)




In [29]:
EPOCH = 200  # train the training data n times, to save time, we just train 1 epoch
graph_name = '字符200，layer=2，dropout=0.5'

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 让torch判断是否使用GPU，建议使用GPU环境，因为会快很多
for epoch in range(101, EPOCH + 1):
    print('epoch:{}'.format(epoch))
    train(model=rnn, device=DEVICE, train_loader=train_loader, optimizer=optimizer, loss_func=criterion, epoch=epoch)

    model_test(model=rnn, device=DEVICE, test_loader=test_loader)

epoch:101
model_test data accuracy: 0.310344828
epoch:102
model_test data accuracy: 0.301724138
epoch:103
model_test data accuracy: 0.310344828
epoch:104
model_test data accuracy: 0.387931034
epoch:105
model_test data accuracy: 0.465517241
epoch:106
model_test data accuracy: 0.405172414
epoch:107
model_test data accuracy: 0.362068966
epoch:108
model_test data accuracy: 0.456896552
epoch:109
model_test data accuracy: 0.465517241
epoch:110
model_test data accuracy: 0.517241379
epoch:111
model_test data accuracy: 0.482758621
epoch:112
model_test data accuracy: 0.594827586
epoch:113
model_test data accuracy: 0.594827586
epoch:114
model_test data accuracy: 0.637931034
epoch:115
model_test data accuracy: 0.517241379
epoch:116
model_test data accuracy: 0.491379310
epoch:117
model_test data accuracy: 0.698275862
epoch:118
model_test data accuracy: 0.732758621
epoch:119
model_test data accuracy: 0.715517241
epoch:120
model_test data accuracy: 0.750000000
epoch:121
model_test data accuracy: 0.75